In [2]:
import pandas as pd
import numpy as np
from usefuls import atx_zip_codes, similar_offense_type
from census import Census

In [2]:
df = pd.read_csv('Crime_Reports.csv')
# Keep only the austin zip codes (this drops the NaN's, 0's and the several typos)
df = df[df['Zip Code'].isin(atx_zip_codes)]
df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,20172571868,AGG ROBBERY/DEADLY WEAPON,300,N,09/14/2017 10:37:00 PM,09/14/2017,2237.0,09/14/2017 10:37:00 PM,09/14/2017,2237.0,...,18.04,O,09/20/2017,120,Robbery,3125442.0,3125442.0,30.333542,-97.705762,"(30.33354183, -97.70576196)"
1,20145045267,BURGLARY OF VEHICLE,601,N,10/11/2014 05:00:00 PM,10/11/2014,1700.0,10/12/2014 04:13:00 AM,10/12/2014,413.0,...,11.00,N,11/04/2014,23F,Theft,3115529.0,3115529.0,30.268006,-97.738955,"(30.26800598, -97.73895531)"
2,20155006575,FORGERY AND PASSING,1000,N,02/11/2015 03:29:00 PM,02/11/2015,1529.0,02/15/2015 03:29:00 PM,02/15/2015,1529.0,...,9.02,N,02/17/2015,NaN,NaN,3121952.0,3121952.0,30.261677,-97.718775,"(30.26167703, -97.71877525)"
3,20182741878,POSS OF DRUG PARAPHERNALIA,1818,N,10/01/2018 11:51:00 PM,10/01/2018,2351.0,10/01/2018 11:51:00 PM,10/01/2018,2351.0,...,17.53,N,10/07/2018,NaN,NaN,3112827.0,3112827.0,30.398291,-97.744022,"(30.39829057, -97.74402237)"
4,20033270128,POSS OF DRUG PARAPHERNALIA,1818,N,11/23/2003 01:06:00 AM,11/23/2003,106.0,11/23/2003 01:06:00 AM,11/23/2003,106.0,...,11.00,C,11/24/2003,NaN,NaN,3116240.0,3116240.0,30.264357,-97.736801,"(30.26435664, -97.73680064)"


In [3]:
mask = ['Zip Code', 'Occurred Date', 'Highest Offense Description']
mdf = df[mask]
# Convert zip codes to integer type
mdf['Zip Code'] = mdf['Zip Code'].apply(lambda x: int(x))
# Extract only years from date
mdf['year'] = mdf['Occurred Date'].apply(lambda d : int(d[-4:]))
# Only look at years 2009 onwards
mdf = mdf[mdf.year >= 2009]
mdf = mdf[mdf.year <= 2018]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
# Combine similar offenses
mdf["Highest Offense Description"] = mdf["Highest Offense Description"].replace(similar_offense_type)

Other                  704386
Theft                  194321
Burglary               180907
Aggravated Assault     128625
Motor Vehicle Theft     20880
Sexual Assault          11897
Robbery                  9948
Arson                     901
Murder                    272
Manslaughter               51
Negligent Homicide          5
Name: Highest Offense Description, dtype: int64

In [10]:
atx_crime_types_df = mdf.groupby(['year', 'Zip Code', 'Highest Offense Description']).count()
atx_crime_types_df = atx_crime_types_df.rename(columns={'Occurred Date': 'Reports'})

In [16]:
## Now we are collecting number of reports of each type of crime per year

type_years = mdf.groupby(['year', 'Highest Offense Description']).count().drop(labels='Zip Code', axis=1)

In [20]:
type_years = type_years.rename(columns={'Occurred Date': 'Reports'})

In [36]:
type_years

Reports
year Highest Offense Description         
2009 Aggravated Assault             13436
     Arson                             80
     Burglary                       25993
     Manslaughter                       6
     Motor Vehicle Theft             2057
     Murder                            21
     Other                          86546
     Robbery                         1378
     Sexual Assault                  1344
     Theft                          20324
2010 Aggravated Assault             12897
     Arson                            110
     Burglary                       23414
     Manslaughter                       6
     Motor Vehicle Theft             2113
     Murder                            33
     Other                          84322
     Robbery                         1196
     Sexual Assault                  1256
     Theft                          20997
2011 Aggravated Assault             13331
     Arson                            117
     Burglary                       20047
     Manslaughter                       1
     Motor Vehicle Theft             1927
     Murder                            26
     Negligent Homicide                 1
     Other                          77638
     Robbery                         1066
     Sexual Assault                  1145
...                                   ...
2016 Burglary                       13650
     Manslaughter                       7
     Motor Vehicle Theft             2010
     Murder                            35
     Negligent Homicide                 1
     Other                          63332
     Robbery                          963
     Sexual Assault                  1263
     Theft                          18373
2017 Aggravated Assault             12522
     Arson                             96
     Burglary                       13563
     Manslaughter                       3
     Motor Vehicle Theft             1995
     Murder                            25
     Negligent Homicide                 1
     Other                          62163
     Robbery                          926
     Sexual Assault                  1360
     Theft                          15864
2018 Aggravated Assault             11985
     Arson                             64
     Burglary                       14722
     Manslaughter                       6
     Motor Vehicle Theft             2421
     Murder                            27
     Other                          53492
     Robbery                          991
     Sexual Assault                  1140
     Theft                          16084

[104 rows x 1 columns]

In [1]:
.37**2

0.1369